In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

## The crawling phase:
## We did the crawling on the english's teachers on Preply web

In [2]:
# in this arr we get the links of specific teacher
urls = []
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}


## we download all the pages that contains cards of English's teachers 
## each page contains 10 cards,and every card represents specific english teacher
## we append every card to "urls" array.

In [3]:
reviews = []
student_count = []
for i in range(477,487):
    url = f'https://preply.com/en/online/english-tutors?page={i}'
    res = requests.get(url,headers=headers)
    soup = BeautifulSoup(res.content,'html.parser')
    all_cards = soup.find_all('li',attrs={"class":"TutorCardWrapper___2SCl8"})
    for card in all_cards:
        href = card.find('a')['href']
        urls.append(href)
        try:
            reviews.append(card.find('div',attrs={"class":"ReviewsNumber___1enrU"}).text)
        except:
            reviews.append(0)
        try:
            student_counts = card.find('span',attrs={"class":"ActiveStudents___I9k62"}).text
            student_count.append(int(student_counts.split(' ')[0]))
        except:
            student_count.append(0)


In [4]:
len(urls)

100

## here we created the columns which represntes the features of the the teacher on our data frame.

In [5]:
  to_df = {
    "name":[],
    "where_from":[],
    "description":[],
    "price":[],
    "teaches":[],
    "lessons":[],
    "rating":[],
    "speaks":[],
    "more_info":[],
    "popular":[],
    "newly_joined":[],
    "responds":[],
    "has_diploma":[],
    "approved_matirials":[],
    "reviews":reviews,
    "students_count":student_count
}
t_soup = None

## Crawling to urls:
### after we got the information of every teacher on preply web, We inserted them to our data farme. 

In [ ]:
# range(i) ---> how many teachers from urls[]
for u in range(100):
    speaks = []
    teaches = []
    res = requests.get(urls[u],headers=headers)
    
    t_soup = BeautifulSoup(res.content,'html.parser')
    
    name = t_soup.find('h2',attrs={"class":'name___10LsT'}).text
    where_from=t_soup.find('img',attrs={"class":'flag___26DQj'})['alt']
    description = t_soup.find('h1',attrs={"class":'infoRow___dv67Z headline___3KNBD'}).text
    
    teaches_arr = t_soup.find_all('li',attrs={"class":'item___2xHv5'})
    
    newly_joined = 0
    has_diploma=0
    approved_matirials=0
    
    for l in teaches_arr:
        teaches.append(l.find('a').text)

    speaks_arr = t_soup.find_all('li',attrs={"class":'item___18Wix'})
    
    for s in speaks_arr:
        speaks.append(s.text)


    try:
        rating = t_soup.find('div',attrs={"class":'RatingIndicatorRating___374zP'}).text
    except:
        rating = 0
        
    try:
        popular = t_soup.find('p',attrs={"class":'flexColumn___596kL'}).find('span',attrs={"class":"bold___14vuF"}).text
    except:
        popular = 0
        
        
    try:
        t_soup.find('div',attrs={"class":"NewTutorBadge___sVfwo"}).text
        newly_joined=1
    except:
        newly_joined = 0
        
    try:   
        lessons = t_soup.find('span',attrs={"class":"totalLessons___2fSGl"}).text
    except:
        lessons=0
        
        
    price = t_soup.find('div',attrs={"class":'PriceIndicatorValue___1kV4p'}).text
    
    try:
        more_info = t_soup.find_all('div',attrs={"class":'MoreInfoItem___2Tprx'})
        to_df['more_info'].append(more_info[1].text)   
    except:
        to_df['more_info'].append(np.nan)   

        
    try:
        more_info = t_soup.find_all('div',attrs={"class":'MoreInfoItem___2Tprx'})
        to_df['responds'].append(more_info[2].text)   
    except:
        to_df['responds'].append(np.nan) 
        
    try:
        t_soup.find('span',attrs={"class":"diploma___1yB6Y"}).text
        has_diploma=1
        
        
    except:
        has_diploma=0
        
    
    try:
        t_soup.find('b',attrs={"class":"curriculumBold___3mLEI"}).text
        approved_matirials=1
        
    except:
        approved_matirials=0
    
    sp = ','.join(speaks)
    tc = ','.join(teaches)
    to_df['name'].append(name)
    to_df['where_from'].append(where_from)
    to_df['description'].append(description)    
    to_df['teaches'].append(tc)
    to_df['speaks'].append(sp)    
    to_df['lessons'].append(lessons)    
    to_df['price'].append(price)
    to_df['popular'].append(popular)
    to_df['rating'].append(rating) 
    to_df['newly_joined'].append(newly_joined)
    to_df['has_diploma'].append(has_diploma)
    to_df['approved_matirials'].append(approved_matirials)

### As you can see we had a lot a "try and except" because the pages were uneven.
#### for example: 
1. some people didnt have rating, as a result they didnt have the class that contains the rating information as other people that has that.
2. there are some people that called "popular" because their features.. so they have on their card a script that says that. people who are not popular do not have it.
3. people that just joind preply, have on their cards a bold script that says that, other people don't.
4. there some people that wrote on their card "More Info", and other people mdont..
the same happens on the "has_diploma","approved_matirials","responds" and "lessons" categories and respectively their classes.

In [ ]:
df = pd.DataFrame.from_dict(to_df)

In [ ]:
df.info()

In [ ]:
df.to_csv('preply17.csv',sep=',')